In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import psycopg
from trino.dbapi import connect
import pandas as pd

# this is to better display pyspark and pandas dataframes
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

pd.set_option('display.max_colwidth', None)

In [13]:
spark = SparkSession.builder \
  .appName("iceberg-rest") \
  .config("spark.driver.memory", "4g") \
  .config("spark.executor.memory", "4g") \
  .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
  .config("spark.jars", "/opt/extra-jars/iceberg-spark-runtime.jar,/opt/extra-jars/iceberg-aws-bundle.jar") \
  .config(f"spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog") \
  .config(f"spark.sql.catalog.iceberg.type", "rest") \
  .config(f"spark.sql.catalog.iceberg.uri", "http://iceberg-rest-catalog:8000") \
  .config(f"spark.sql.catalog.iceberg.io-impl", "org.apache.iceberg.aws.s3.S3FileIO") \
  .config(f"spark.sql.catalog.iceberg.warehouse", "s3://warehouse/iceberg/") \
  .config(f"spark.sql.catalog.iceberg.s3.endpoint", "http://minio:9000") \
  .config(f"spark.sql.catalog.iceberg.s3.path-style-access", "true") \
  .getOrCreate()

25/04/25 14:08:02 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [14]:
print("Spark Version: ", spark.sparkContext.version)
print("Scala Version: ", spark.sparkContext._jvm.scala.util.Properties.versionString())

Spark Version:  3.5.5
Scala Version:  version 2.13.8


In [19]:
spark.sql("SHOW TABLES IN iceberg.icebergdata").show()

+-----------+--------------------+-----------+
|  namespace|           tableName|isTemporary|
+-----------+--------------------+-----------+
|icebergdata|                dbz_|      false|
|icebergdata|dbz__main_dbo_Cus...|      false|
|icebergdata|debezium_database...|      false|
|icebergdata|debezium_offset_s...|      false|
+-----------+--------------------+-----------+



In [28]:
spark.sql("select * from iceberg.icebergdata.dbz__main_dbo_Customers").show()

+----------+-----------+--------+--------------------+--------------------+--------------------+---------+----+---------+-------------------+----+
|CustomerID|  FirstName|LastName|               Email|         CreatedDate|         UpdatedDate|__deleted|__op|  __table|     __source_ts_ns|__db|
+----------+-----------+--------+--------------------+--------------------+--------------------+---------+----+---------+-------------------+----+
|        33|       Demo|Anderson|thomas.anderson@e...|2025-04-03 14:05:...|2025-04-03 14:05:...|    false|   u|Customers|1745590919697000000|main|
|        32|Hello World|  Wilson|emma.wilson@examp...|2025-04-03 14:05:...|2025-04-03 14:05:...|    false|   u|Customers|1745590705380000000|main|
|        12|       John|   Smith|john.smith@exampl...|2025-03-31 12:06:...|2025-03-31 12:06:...|    false|   r|Customers|1745589896781180600|main|
|        13|       Jane|     Doe|jane.doe@example.com|2025-03-31 12:06:...|2025-03-31 12:06:...|    false|   r|Custome

In [ ]:
# # Connect Trino
# trino_conn = connect(
#     host="trino",
#     port=8080,
#     user="user"
# )

# df_from_trino = pd.read_sql_query('select * from "iceberg-jdbc".jdbc.yellow_tripdata limit 10', conn)
# df_from_trino